<a href="https://colab.research.google.com/github/lelongc/ro/blob/main/cube3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Cube 3D

In [1]:
!git clone https://github.com/Roblox/cube

Cloning into 'cube'...
remote: Enumerating objects: 217, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 217 (delta 78), reused 50 (delta 44), pack-reused 104 (from 1)
Receiving objects: 100% (217/217), 40.62 MiB | 39.28 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [2]:
%cd /content/cube

/content/cube


In [3]:
!huggingface-cli download Roblox/cube3d-v0.5 --local-dir ./model_weights

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 6 files:   0% 0/6 [00:00<?, ?it/s]Downloading 'assets/bbox_conditioning.gif' to 'model_weights/.cache/huggingface/download/assets/Wt6CXn0yhR4MhM-huE19SfAVTrs=.784405469cc3986d2a8a22862487121d223e84de3281b7bc61722d092b1f8cd4.incomplete'

.gitattributes: 1.56kB [00:00, 7.31MB/s]
Download complete. Moving file to model_weights/.gitattributes
Fetching 6 files:  17% 1/6 [00:00<00:03,  1.59it/s]
README.md: 2.64kB [00:00, 4.82MB/s]
Download complete. Moving file to model_weights/README.md

assets/3d_composition.gif:   0% 0.00/9.79M [00:00<?, ?B/s]

shape_gpt.safetensors:   0% 0.00/7.17G [00:00<?, ?B/s]


shape_tokenizer.safetensors:   0% 0.00/1.10G [00:00<?, ?B/s]



assets/bbox_conditioning.gif:   0% 0.00/808k [00:00<?, ?B/s]



assets/bbox_conditioning.gif: 100% 808k/808k [00:00<00:00, 1.58MB/s]
Download complete. Moving file to model_weights/assets/bbox_conditioning.gif

assets/3d_composition.gif: 10

In [4]:
!pip install trimesh omegaconf warp-lang transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.0/737.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 MB 8.2 MB/s eta 0:00:00


In [5]:
%cd /content/cube

/content/cube


# Create Inference Engine

In [6]:
from cube3d.inference.engine import Engine
import torch
# load ckpt
config_path = "cube3d/configs/open_model_v0.5.yaml"
gpt_ckpt_path = "model_weights/shape_gpt.safetensors"
shape_ckpt_path = "model_weights/shape_tokenizer.safetensors"
engine = Engine(
    config_path,
    gpt_ckpt_path,
    shape_ckpt_path,
    device=torch.device("cuda"),
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

# Generate a Mesh from text

In [9]:
# input_prompt = "A tall pagoda"
# # Use a lower resolution_base to accomodate limited GPU VRAM on Colab notebooks
# mesh_v_f = engine.t2s([input_prompt], use_kv_cache=True, resolution_base=5.0, bounding_box_xyz=(1.5, 2.0, 1.0))

In [10]:
# Copy đoạn này thay vào chỗ input_prompt cũ
input_prompt = "A low poly medieval iron sword with a brown leather handle"
# Chỉnh lại dòng này trong code của bạn để kiếm không bị mập
mesh_v_f = engine.t2s([input_prompt], use_kv_cache=True, resolution_base=5.0, bounding_box_xyz=(0.5, 2.0, 0.5))

extracting geometry: 100%|██████████| 1/1 [00:00<00:00,  5.89chunk/s]


# Visualize the output

In [12]:
# Visualize the model
import plotly.graph_objects as go
vertices = mesh_v_f[0][0]
faces = mesh_v_f[0][1]
fig = go.Figure(
    data=[
        go.Mesh3d(
            x=vertices[:,0],
            y=vertices[:,1],
            z=vertices[:,2],
            i=faces[:,0],
            j=faces[:,1],
            k=faces[:,2],
            opacity=1.0)
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()

In [13]:
import trimesh
from google.colab import files
import numpy as np

# 1. Lấy dữ liệu từ kết quả của model
# (mesh_v_f chứa vertices và faces)
vertices = mesh_v_f[0][0]
faces = mesh_v_f[0][1]

# 2. Chuyển đổi từ Tensor sang Numpy (nếu cần) để xử lý
try:
    if hasattr(vertices, 'cpu'):
        vertices = vertices.cpu().numpy()
    if hasattr(faces, 'cpu'):
        faces = faces.cpu().numpy()
except:
    pass

# 3. Tạo đối tượng Mesh bằng thư viện Trimesh
mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

# 4. Lưu thành file .obj
file_name = "roblox_model.obj"
mesh.export(file_name)
print(f"Đã lưu file: {file_name}")

# 5. Tự động tải xuống máy tính của bạn
files.download(file_name)

Đã lưu file: roblox_model.obj


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>